In [ ]:
import json
from typing import List
from sentence_transformers import SentenceTransformer
import lancedb
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry
import os
import shutil

In [ ]:
with open('../data/index_items.json', 'r') as file:
    entities = json.load(file)

In [ ]:
lancedb_path = "../.lancedb"

Delete the .lancedb folder before running the code:

In [ ]:
if os.path.exists(lancedb_path) and os.path.isdir(lancedb_path):
    # Remove the directory and all its contents
    shutil.rmtree(lancedb_path)
    print(f"The directory '{lancedb_path}' has been removed.")
else:
    print(f"The directory '{lancedb_path}' does not exist.")

In [ ]:
db = lancedb.connect(lancedb_path)
title_model = get_registry().get("sentence-transformers").create(name="BAAI/bge-small-en-v1.5", device="cpu")
summary_model = get_registry().get("sentence-transformers").create(name="BAAI/bge-small-en-v1.5", device="cpu")

In [ ]:
class Words(LanceModel):
    title: str = title_model.SourceField()
    title_vector: Vector(title_model.ndims()) = title_model.VectorField() 
    summary: str = summary_model.SourceField()
    summary_vector: Vector(summary_model.ndims()) = summary_model.VectorField() 
    section: str
    page: int
    wiki: bool
    title_index: int
    instance_of: List[str]
    wikipedia_url: str
    wikidata_id: str

In [ ]:
entity_table = db.create_table("entities", schema=Words)
entity_table.add(
    entities
)

### Load

In [ ]:
db = lancedb.connect(lancedb_path)

In [ ]:
table = db.open_table("entities")